In [2]:
print("OmriShemTov Hatich Shel Yeled  - Ground Truth")

OmriShemTov Hatich Shel Yeled  - Ground Truth


## Setup

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE ##TODO: Insert to ENV.yml
from imblearn.under_sampling import RandomUnderSampler
import statsmodels.api as sm
import random
seed=42
random.seed(seed)
from xgboost import XGBClassifier
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

## Preprocess function

- create frequency columns for some lab variables (??)
- crop time_bm (10) last ICULOS hours for each patient
- handle Units1/2/3
- apply aggregations

In [9]:
def lr_preprocess_sepsis(df, window_size=5, time_bm=-10):

    # columns for using
    frequency_used_attributes = ['BaseExcess',  'FiO2', 'pH', 'PaCO2', 'Glucose','Lactate', 'PTT']
    values_used_attributes = [ 'Hct', 'Glucose','Potassium']
    constant_attributes = ['ID','max_ICULOS', 'Gender']
    other_attributes = ['time_bm','HR','MAP','O2Sat', 'Resp','SBP', 'ICULOS']
    units_attributes = ['Unit1', 'Unit2']
    label_attributes= ['Label','SepsisLabel']

    ## TO DELETE (not used)
    # def add_frequency(df, freq_columns):
    #     df = df.sort_values(by=['ID','ICULOS'], ascending =[True,True])
    #     rolling = df[['ID','max_ICULOS']+self.freq_columns].groupby(by=['ID','max_ICULOS'])[freq_columns].expanding().count().reset_index()
    #     for at in freq_columns:
    #         rolling[at] = rolling[at]/rolling['max_ICULOS']
    #     rolling= rolling.rename(columns={at: f'freq_{at}' for at in freq_columns})
    #     combined = df.join(rolling,how='left',on='ID', rsuffix= 'r')
    #     return combined
    # #apply
    # normalized_frequency_attr = ['FiO2', 'pH', 'PaCO2', 'Glucose','Lactate'] #choose attributes
    # add_frequency(df, normalized_frequency_attr)
    # normalized_frequency_attr = [f'freq_{at}' for at in normalizes_frequency_attr] #fix names

    # create frequency columns for some lab variables
    def add_rolling_window(df, attr, window_size):
        df = df.sort_values(by=['ID','ICULOS'], ascending =[True,True])
        rolling = df[['ID']+attr].groupby('ID').rolling(window=window_size, closed='both').count()
        rolling= rolling.rename(columns={at: f'{window_size}w_sum_{at}' for at in attr})
        rolling=rolling[list(rolling.columns)[1:]].reset_index().set_index('level_1')
        combined = df.join(rolling,how='left', rsuffix= 'r')
        return combined, rolling
    #apply
    df_with_roll, df_roll = add_rolling_window(df,frequency_used_attributes,window_size)
    frequency_used_attributes_fixed = [f'{window_size}w_sum_{x}' for x in frequency_used_attributes]
    df = df_with_roll


    # crop 10 (time_bm) last ICULOS hours for each patient
    df = df[df['time_bm']>=time_bm]

    # handle Units123
    df['Unit3'] = ( (1*(df['Unit1']+df['Unit2'])<1) |
                          (df['Unit1'].isna() & df['Unit2'].isna()) )*1
    df['Unit1'][df['Unit1'].isna()] = 0
    df['Unit2'][df['Unit2'].isna()] = 0
    df[['Unit1','Unit2','Unit3']]

    # aggregations
    data_final = df.groupby(['ID', 'Label','max_ICULOS','Gender']).agg({
                                                        'Unit1': 'max',\
                                                        'Unit2': 'max',\
                                                        'Unit3': 'max',\
                                                        'HR': ['median', 'max'],\
                                                        'MAP': ['median', 'min'],\
                                                        'O2Sat': ['mean'],\
                                                        'Resp': ['median', 'max'],\
                                                        'SBP': ['median', 'min'],\
                                                        'Hct': ['median', 'min'],\
                                                        'Potassium': 'mean',\
                                                        'Glucose': 'mean',\
                                                        'Temp': ['mean', 'min'],\
                                                        'DBP': 'mean',\
                                                        'WBC': ['median', 'min'],\
                                                        'EtCO2': 'mean',\
                                                        'BaseExcess': 'mean',\
                                                        'HCO3': 'mean',\
                                                        'FiO2': 'mean',\
                                                        'SaO2': 'mean',\
                                                        'AST': 'mean',\
                                                        'Lactate': 'mean',\
                                                        'Magnesium': 'mean',\
                                                        'Phosphate': 'mean',\
                                                        'TroponinI': 'mean',\
                                                        'Hgb': 'mean',\
                                                        'PTT': 'mean',\
                                                        'Platelets': 'mean',\
                                                        'Age': 'mean',\
                                                        'HospAdmTime': 'mean',\
                                                        f'{window_size}w_sum_BaseExcess': 'mean',\
                                                        f'{window_size}w_sum_FiO2': 'mean',\
                                                        f'{window_size}w_sum_pH': 'mean',\
                                                        f'{window_size}w_sum_PaCO2': 'mean',\
                                                        f'{window_size}w_sum_Glucose': 'mean',\
                                                        f'{window_size}w_sum_Lactate': 'mean',\
                                                        f'{window_size}w_sum_PTT': 'mean'}).reset_index()
    data_final.columns = ['__'.join(col).strip() for col in data_final.columns.values]
    data_final.rename(columns={"ID__": "ID", "Label__": "Label", "max_ICULOS__":"max_ICULOS", "Gender__":"Gender"}, inplace=True)

    return data_final


## Imutation functions

In [10]:
def imputation_with_KNNimputer(train_df, n=3):
    data_knn_imputed = train_df.copy(deep=True)    # Copy the data
    knn_imp = KNNImputer(n_neighbors=n) # Init the transformer
    knn_imp.fit(data_knn_imputed)
    return knn_imp #return fitted inputer

## Oversampling using SMOTE (function)

In [11]:
def os_with_smote(train_df, p=0.5):
    X = train_df.loc[:, train_df.columns != 'Label']
    y = train_df.loc[:, train_df.columns == 'Label']
    os = SMOTE(sampling_strategy=p, random_state=0)
    columns = X.columns
    os_data_X, os_data_y = os.fit_resample(X, y)
    os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
    os_data_y = pd.DataFrame(data=os_data_y,columns=['Label'])
    return os_data_X, os_data_y

## Load data

In [12]:
train_df = pd.read_csv("/home/student/filtered_train_df_0705.csv")
val_df = pd.read_csv("/home/student/filtered_val_df_0705.csv")
test_df =  pd.read_csv("/home/student/filtered_test_df_0705.csv")

## Handle training data

In [13]:
train_df = lr_preprocess_sepsis(train_df)
# Check the numbers of our data
print("length of train data is",len(train_df))
print("Number of Label=0 in train data",len(train_df[train_df['Label']==0]))
print("Number of Label=1 in train data",len(train_df[train_df['Label']==1]))
print("Proportion of Label=0 data in train data is ",len(train_df[train_df['Label']==0])/len(train_df))
print("Proportion of Label=1 data in train data is ",len(train_df[train_df['Label']==1])/len(train_df))

/data/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py:2010: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
<ipython-input-9-e0d7d5e87270>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unit3'] = ( (1*(df['Unit1']+df['Unit2'])<1) |
<ipython-input-9-e0d7d5e87270>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unit1'][df['Unit1'].isna()] = 0
/data/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: Sett

length of train data is 16000
Number of Label=0 in train data 14857
Number of Label=1 in train data 1143
Proportion of Label=0 data in train data is  0.9285625
Proportion of Label=1 data in train data is  0.0714375


## TODO: Choose final columns and create X,y

### Fit&Transform imputation (TODO: SAVE IMPUTER)

In [14]:
# imputing training data
# TODO: Save Imputer
knn_imp = imputation_with_KNNimputer(train_df, 3)
train_df.loc[:, :] = knn_imp.transform(train_df)

### Handle Imbalance

In [15]:
# over-sampling training data
X, y = os_with_smote(train_df, p=0.25)

# Check the numbers of our data
print("length of oversampled data is ",len(X))
print("Number of Label=0 in oversampled data",len(y[y['Label']==0]))
print("Number of Label=1",len(y[y['Label']==1]))
print("Proportion of Label=0 data in oversampled data is ",len(y[y['Label']==0])/len(X))
print("Proportion of Label=1 data in oversampled data is ",len(y[y['Label']==1])/len(X))

length of oversampled data is  18571
Number of Label=0 in oversampled data 14857
Number of Label=1 3714
Proportion of Label=0 data in oversampled data is  0.8000107694792957
Proportion of Label=1 data in oversampled data is  0.19998923052070433


In [16]:
#Downsample majority class
under = RandomUnderSampler(sampling_strategy=0.5)
X,y = under.fit_resample(X, y)

# Check the numbers of our data
print("length of final training data is ",len(X))
print("Number of Label=0 in final training data",len(y[y['Label']==0]))
print("Number of Label=1",len(y[y['Label']==1]))
print("Proportion of Label=0 data in final data training is ",len(y[y['Label']==0])/len(X))
print("Proportion of Label=1 data in final training data is ",len(y[y['Label']==1])/len(X))

length of final training data is  11142
Number of Label=0 in final training data 7428
Number of Label=1 3714
Proportion of Label=0 data in final data training is  0.6666666666666666
Proportion of Label=1 data in final training data is  0.3333333333333333


## Evaluation data (preprocess)

In [17]:
val_df = lr_preprocess_sepsis(val_df)
val_df.loc[:, :] = knn_imp.transform(val_df)

test_df = lr_preprocess_sepsis(test_df)
test_df.loc[:, :] = knn_imp.transform(test_df)

/data/anaconda3/lib/python3.8/site-packages/pandas/core/window/rolling.py:2010: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
<ipython-input-9-e0d7d5e87270>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unit3'] = ( (1*(df['Unit1']+df['Unit2'])<1) |
<ipython-input-9-e0d7d5e87270>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unit1'][df['Unit1'].isna()] = 0
/data/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: Sett

In [18]:
X_val = val_df.loc[:, val_df.columns != 'Label']
y_val = val_df.loc[:, val_df.columns == 'Label']

X_test = test_df.loc[:, test_df.columns != 'Label']
y_test = test_df.loc[:, test_df.columns == 'Label']

## Models

In [19]:
logit_model = sm.Logit(y,X)
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.543233
         Iterations 6
                              Results: Logit
Model:                  Logit               Pseudo R-squared:    0.147     
Dependent Variable:     Label               AIC:                 12197.4039
Date:                   2022-05-10 11:31    BIC:                 12534.0539
No. Observations:       11142               Log-Likelihood:      -6052.7   
Df Model:               45                  LL-Null:             -7092.0   
Df Residuals:           11096               LLR p-value:         0.0000    
Converged:              1.0000              Scale:               1.0000    
No. Iterations:         6.0000                                             
---------------------------------------------------------------------------
                         Coef.   Std.Err.    z     P>|z|   [0.025   0.975] 
---------------------------------------------------------------------------
ID               

In [20]:
logreg = LogisticRegression()
logreg.fit(X, y)

/home/student/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/student/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
y_pred = logreg.predict(X_val)

In [22]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_val, y_val)))
print(f"exact: {(val_df['Label']==y_pred).mean()}")

Accuracy of logistic regression classifier on test set: 0.90
exact: 0.9015


In [23]:
(val_df['Label']==y_pred).mean()

0.9015

In [24]:
from sklearn.metrics import f1_score

print('Val F1: ', f1_score(y_val, y_pred, average='macro'))

# print(f1_score(y_val, y_pred, average='micro'))
# print(f1_score(y_val, y_pred, average='weighted'))
# print(f1_score(y_val, y_pred, average=None))

Val F1:  0.5889256143162392


In [25]:
xgbc = XGBClassifier()
xgbc.fit(X, y)
y_train_pred = xgbc.predict(X)
print(f'Train F1: {f1_score(y,y_train_pred)}')
y_val_pred = xgbc.predict(X_val)
print(f'Val F1: {f1_score(y_val,y_val_pred)}')
y_test_pred = xgbc.predict(X_test)
print(f'Test F1: {f1_score(y_test,y_test_pred)}')

Train F1: 1.0
Val F1: 0.7382812499999999
Test F1: 0.7024734982332156


In [26]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100, random_state=0)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X,y)
y_train_pred = clf.predict(X)
print(f'Train F1: {f1_score(y, y_train_pred)}')
y_val_pred = clf.predict(X_val)
print(f'Val F1: {f1_score(y_val, y_val_pred)}')
y_test_pred = clf.predict(X_test)
print(f'Test F1: {f1_score(y_test, y_test_pred)}')

<ipython-input-26-a4ee45e6f201>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X,y)


Train F1: 1.0
Val F1: 0.7125984251968502
Test F1: 0.6879073135409125


In [27]:
# clf.summary()